In [1]:
from transformers import pipeline

In [2]:
# 가려진 부분을 채워넣기
nlp = pipeline("fill-mask")

No model was supplied, defaulted to distilroberta-base (https://huggingface.co/distilroberta-base)
Downloading: 100%|██████████| 480/480 [00:00<00:00, 480kB/s]
Downloading: 100%|██████████| 465M/465M [00:23<00:00, 20.4MB/s] 
All model checkpoint layers were used when initializing TFRobertaForMaskedLM.

All the layers of TFRobertaForMaskedLM were initialized from the model checkpoint at distilroberta-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForMaskedLM for predictions without further training.
Downloading: 100%|██████████| 878k/878k [00:01<00:00, 719kB/s]  
Downloading: 100%|██████████| 446k/446k [00:00<00:00, 465kB/s]  
Downloading: 100%|██████████| 1.29M/1.29M [00:01<00:00, 961kB/s] 


In [5]:
text = f"Pizza is my {nlp.tokenizer.mask_token}"
text

'Pizza is my <mask>'

In [7]:
nlp(text)

[{'score': 0.07068377733230591,
  'token': 2674,
  'token_str': ' favorite',
  'sequence': 'Pizza is my favorite'},
 {'score': 0.0355093739926815,
  'token': 4592,
  'token_str': ' lunch',
  'sequence': 'Pizza is my lunch'},
 {'score': 0.03522920608520508,
  'token': 8492,
  'token_str': ' cake',
  'sequence': 'Pizza is my cake'},
 {'score': 0.026076242327690125,
  'token': 9366,
  'token_str': ' pizza',
  'sequence': 'Pizza is my pizza'},
 {'score': 0.02567906863987446,
  'token': 17927,
  'token_str': ' dessert',
  'sequence': 'Pizza is my dessert'}]

In [10]:
text2 = f"Pizza is {nlp.tokenizer.mask_token} my {nlp.tokenizer.mask_token} food"
nlp(text2)

[[{'score': 0.07517623156309128,
   'token': 95,
   'token_str': ' just',
   'sequence': '<s>Pizza is just my<mask> food</s>'},
  {'score': 0.06924442946910858,
   'token': 122,
   'token_str': ' now',
   'sequence': '<s>Pizza is now my<mask> food</s>'},
  {'score': 0.06733357906341553,
   'token': 460,
   'token_str': ' always',
   'sequence': '<s>Pizza is always my<mask> food</s>'},
  {'score': 0.05711856484413147,
   'token': 1153,
   'token_str': ' probably',
   'sequence': '<s>Pizza is probably my<mask> food</s>'},
  {'score': 0.05621125549077988,
   'token': 8127,
   'token_str': ' NOT',
   'sequence': '<s>Pizza is NOT my<mask> food</s>'}],
 [{'score': 0.5211484432220459,
   'token': 2674,
   'token_str': ' favorite',
   'sequence': '<s>Pizza is<mask> my favorite food</s>'},
  {'score': 0.19279128313064575,
   'token': 5863,
   'token_str': ' comfort',
   'sequence': '<s>Pizza is<mask> my comfort food</s>'},
  {'score': 0.15253256261348724,
   'token': 5548,
   'token_str': ' fav

In [11]:
from transformers import TFAutoModelForMaskedLM, AutoTokenizer

In [12]:
tokenizer = AutoTokenizer.from_pretrained("distilroberta-base")
model = TFAutoModelForMaskedLM.from_pretrained("distilroberta-base")

All model checkpoint layers were used when initializing TFRobertaForMaskedLM.

All the layers of TFRobertaForMaskedLM were initialized from the model checkpoint at distilroberta-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForMaskedLM for predictions without further training.


In [13]:
vocab = tokenizer.get_vocab()
id2word = {i:word for word,i in vocab.items()}

In [14]:
sequence = f"Pizza is my {tokenizer.mask_token} food"
sequence

'Pizza is my <mask> food'

In [15]:
input_ids = tokenizer.encode(sequence, return_tensors="tf")
input_ids

<tf.Tensor: shape=(1, 8), dtype=int32, numpy=array([[    0,   510, 35280,    16,   127, 50264,   689,     2]])>

In [16]:
tokenizer.mask_token_id

50264

In [17]:
input_ids[0] == tokenizer.mask_token_id

<tf.Tensor: shape=(8,), dtype=bool, numpy=array([False, False, False, False, False,  True, False, False])>

In [18]:
import tensorflow as tf

In [19]:
mask_token_indices = tf.where(input_ids[0] == tokenizer.mask_token_id)[0].numpy().tolist()
mask_token_indices

[5]

In [20]:
result = model(input_ids)
logits = result[0]

In [21]:
i = mask_token_indices[0]
i

5

In [22]:
logits.shape

TensorShape([1, 8, 50265])

In [23]:
mask_token_logits = logits[0,i,:]
mask_token_logits

<tf.Tensor: shape=(50265,), dtype=float32, numpy=
array([-2.8974793 , -4.8479943 ,  3.5865424 , ..., -4.247888  ,
       -4.94939   , -0.17405021], dtype=float32)>

In [24]:
top  = tf.math.top_k(mask_token_logits, k = 10)
top

TopKV2(values=<tf.Tensor: shape=(10,), dtype=float32, numpy=
array([17.242867 , 16.955906 , 16.14506  , 13.9436865, 13.821488 ,
       13.255374 , 13.070421 , 12.692704 , 12.688813 , 12.608028 ],
      dtype=float32)>, indices=<tf.Tensor: shape=(10,), dtype=int32, numpy=
array([ 2674,  5863,  5548,  3366,  6543,  6813,  7080, 17771, 15163,
        5440])>)

In [25]:
for i in top.indices.numpy().tolist():
    print(id2word[i])

Ġfavorite
Ġcomfort
Ġfavourite
Ġdream
Ġsignature
Ġpreferred
Ġbreakfast
Ġstaple
Ġjunk
Ġpassion


In [27]:
import tensorflow as tf

sequence = f"Pizza {tokenizer.mask_token} my {tokenizer.mask_token} food"
input_ids = tokenizer.encode(sequence, return_tensors="tf")
mask_token_indices = tf.where(input_ids[0] == tokenizer.mask_token_id)
mask_token_indices = tf.squeeze(mask_token_indices).numpy().tolist()

In [36]:
result = model(input_ids)
logits = result[0]
for i in mask_token_indices:
    print(f"==={i}===")
    mask_token_logits = logits[0,i,:]
    top = tf.math.top_k(mask_token_logits, k = 10)
    for i in top.indices.numpy().tolist():
        print(id2word[i],end = ",")
    print()

===3===
Ġis,Ġwith,Ġdelivers,Ġand,Ġfor,:,Ġmakes,Ġserves,Ġ&,Ġtastes,
===5===
Ġfavorite,Ġfavourite,Ġcomfort,Ġjunk,Ġbreakfast,Ġlunch,Ġown,Ġsnack,Ġdog,Ġfried,


In [34]:
result

TFMaskedLMOutput(loss=None, logits=<tf.Tensor: shape=(1, 8, 50265), dtype=float32, numpy=
array([[[33.688335  , -5.1455574 , 20.612871  , ...,  0.273535  ,
          2.7475054 , 12.919054  ],
        [ 1.9103746 , -5.8146024 , 16.06207   , ..., -1.8897159 ,
         -0.8535603 ,  1.0776864 ],
        [ 2.9016135 , -5.552311  ,  8.874351  , ..., -5.091152  ,
         -2.465067  ,  1.0359651 ],
        ...,
        [-3.8873408 , -5.239915  ,  3.279696  , ..., -6.045577  ,
         -6.1138377 , -0.86565375],
        [-3.0044525 , -6.6731157 ,  7.363603  , ..., -4.8081956 ,
         -5.0538235 , -2.0836086 ],
        [15.128035  , -6.6216936 , 21.003073  , ..., -3.602645  ,
         -1.8064204 ,  5.267201  ]]], dtype=float32)>, hidden_states=None, attentions=None)